# Welcome to this Co-culture analysis notebook

Microfluidic live-cell imaging allows us to precisely quantify the composition of co-cultures. Therefore, we label the individual cells with fluorescence reporters, record fluorescence channels and use this information to tell them apart and characterize their growth behavior.

Therfore, in this notebook, we:

1. Perform segmentation on an MLCI time-lapse sequence
2. Extracting individual cell information, including the fluorescence information
3. Cluster cell detections based on their fluorescence
4. Visualize and characterize the individually labeled cell clusters

Have fun with the analysis 🚀

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
if os.environ.get("JYPN_NO_DEP_INSTALL", None) is None:
    %pip uninstall acia -y
    %pip install acia==0.3.0

    # dependencies for Cellpose segmentation
    %pip uninstall -y cellpose
    %pip install --use-pep517 git+https://www.github.com/mouseland/cellpose.git@8ef88040d9aec85737e12c3f2c2969ecf149f7f0
else:
    print("Running in scaling mode! Do not install requirements!")

In [ ]:
import os
from pathlib import Path
from acia import ureg

# channel of the phase-contrast images (chanel ordering starting with 0...)
phase_contrast_channel = 2

# size of a single pixel in the image
pixel_size = 0.074 * ureg.micrometer

# the imaging interval of the time-lapse sequence
imaging_interval = 15 * ureg.minute

# Limit the number of images to analyze
num_images = None

image_id = "43168.tif"

# use current working directory as default storage folder for outputs
storage_folder = os.getcwd()

In [ ]:
image_id = Path(image_id)

# create the output directory
output_path = Path(storage_folder) / "output/"
output_path.mkdir(parents=True, exist_ok=True)

# make path relative (advantage in video embedding)
output_path_rel = output_path.relative_to(Path(os.getcwd()))

if not image_id.exists():
    !wget -O 43168.tif https://fz-juelich.sciebo.de/s/h3DvJ2ZW8y7r7KF/download

In [ ]:
import torch
import logging

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cuda = torch.cuda.is_available()

if not cuda:
  logging.warning("You are not using GPU computation. Thus the deep learning segmentation might take a while!")
  if IN_COLAB:
    logging.warning("Please go to 'Runtime > Change runtime type' in order to select a GPU based runtime in colab!")

In [ ]:
from pathlib import Path
import tifffile
from acia.segm.local import THWCSequenceSource
import numpy as np
from tqdm.auto import tqdm

image_stack = tifffile.imread(image_id)

# bring the image stack into TxHxWxC (time, height, width, channels) format
source = THWCSequenceSource(image_stack) #(np.moveaxis(image_stack, [1,3], [3, 1]))

# Information about the image stack

In [ ]:
import matplotlib.pyplot as plt

T = source.size_t
C = source.size_c

# display markdown
from IPython.display import Video, Markdown, display
display(Markdown("# Image information"))

table = f"""
| Value    | Content |
| --- | --- |
| Image Path | {image_id} |
| T Size | { T } |
| C Size | { C } |
| Channels | {','.join([f"{c}" for c in range(C)])} |
| Imaging Interval | {imaging_interval} |
| Pixel Size | {pixel_size} |
| Phase-Contrast Channel | {phase_contrast_channel} |
| Image dtype | {image_stack.dtype}
"""

display(Markdown(table))
display(Markdown(f"## Preview of channels"))

t = T // 2

image = source.get_frame(t).raw

fig, ax = plt.subplots(1, C, figsize=(15, 15))
for i, c in enumerate(range(0, C)):       # Channel index starts at 1

    if C > 1:
        loc_ax = ax[i]
    else:
        loc_ax = ax

    loc_ax.imshow(image[...,c], cmap="gray")
    loc_ax.set_title(f"Channel {i}, t: {t}")

if num_images is None:
  num_images = T

plt.tight_layout()

# 1. Cell Segmentation

Now we specify the segmentation model: [Omnipose](https://doi.org/10.1101/2021.11.03.467199) and apply it to the phase-contrast channel of the time-lapse.

In [ ]:
import torch
from acia.segm.processor.cellpose_sam import CellposeSAMSegmenter

# connect to remote machine learning model
model = CellposeSAMSegmenter()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# perform overlay prediction
print("Perform Prediction...")
with torch.no_grad():
  result = model(source.to_channel(phase_contrast_channel))

# Visualize the video sequence

1. Place timestamps and scalebar
2. Render segmentation overlay into video

In [ ]:
import acia
from acia.segm.output import renderVideo
from acia.viz import render_segmentation_mask, render_video, render_time, render_scalebar
import numpy as np
from acia import ureg

video_config = dict(codec="vp9", ffmpeg_params = ["-crf", "30", "-b:v", "0", "-speed", "1"])


scalebar_config = dict(
    xy_position=(650, 750),
    size_of_pixel = pixel_size,
    bar_width=10 * ureg.micrometer, # width of the scalebar
    bar_height="1 micrometer" # height of the scalebar
)

time_config = dict(
    xy_position=(670, 20),
    timepoints=np.array(range(num_images)) * imaging_interval, # timepoints of the individual frames (with correct unit)
    background_color = None #(0, 0, 0),
)

To validate the segmentation result, we create a short video:

In [ ]:
framerate=2

# Make a video with
video_file = str(output_path_rel / "segmented.mp4")

source_time = render_time(source.to_channel(phase_contrast_channel).to_rgb(), **time_config)
source_scalebar = render_scalebar(source_time, **scalebar_config)
source_segm = render_segmentation_mask(source_scalebar, result, alpha=0.5)
render_video(source_segm, filename=video_file, **video_config, framerate=framerate)


# Display the rendered segmentation
from IPython.display import Video, Markdown, display
display(Markdown("# Your segmentation"))

from moviepy.editor import *
myvideo =  VideoFileClip(video_file)
myvideo.ipython_display(maxduration=400)

# 2. Extracting individual cell properties

Now that we have the cell segmentation, we can move on and extract individual cell properties like Area, Time, Length, ....
and visualize them in a table:

In [ ]:
from acia.analysis import ExtractorExecutor, AreaEx, IdEx, FrameEx, TimeEx, LengthEx, FluorescenceEx
import pint

ex = ExtractorExecutor()

df = ex.execute(result, source, [
    FrameEx(),
    TimeEx(input_unit=imaging_interval),  # one picture every 15 minutes
    # extract average fluorescence values per cell
    #FluorescenceEx(channels=[1,], channel_names=["e2-crimson",], parallel=1),
    FluorescenceEx(channels=[0,1], channel_names=["e2-crimson", "mvenus"], parallel=1),
    # extract sum of fluorescence per cell
    FluorescenceEx(channels=[0,1], channel_names=["e2-crimson_sum", "mvenus_sum"], summarize_operator=np.sum, parallel=1),
    # define the cell properties that you want to extract here
    AreaEx(input_unit=pixel_size ** 2),  # pass the correct area of pixels
    #LengthEx(input_unit=pixel_size),  # pass the correct size of pixels
])

df

# Filter and Cluster by fluorescences

In [ ]:
# set minimum threshold to filter out background particles
thr_e2_crimson = 420
thr_mvenus = 440

# cells should be bright in at least one fluorescence channel
df = df[((df["mvenus"] > thr_mvenus) | (df["e2-crimson"] > thr_e2_crimson)) & (df["area"] > 1) & (df["area"] < 10)]

In [ ]:
from sklearn.cluster import KMeans, OPTICS, DBSCAN, SpectralClustering, Birch, AgglomerativeClustering
import numpy as np
import seaborn as sns

X = np.array([df["mvenus"].to_numpy(), df["e2-crimson"].to_numpy()]).T

kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X)
labels_ = kmeans.labels_
#labels_ = np.where(X[:,1] > 420, 0, 1)

sns.scatterplot(df, x="mvenus", y="e2-crimson", hue=labels_)
sns.kdeplot(df, x="mvenus", y="e2-crimson", hue=labels_, fill=True)

mvenus_mean_0 = np.mean(X[labels_==0, 0])
mvenus_mean_1 = np.mean(X[labels_==1, 0])

mvenus_label = 1 if mvenus_mean_1 > mvenus_mean_0 else 0
e2_crimson_label = 1 - mvenus_label

In [ ]:
df.loc[:,"label"] = labels_

In [ ]:
df.loc[df.label == e2_crimson_label, "label_name"] = "e2_crimson"
df.loc[df.label == mvenus_label, "label_name"] = "mvenus"

# 3. Use fluorescence to filter artifacts

Non-fluorescent particles are artifacts in this experiment. Thus, we remove them all.

And now let's look at the new video with filtered content

In [ ]:
from acia.base import Overlay
from acia.segm.formats import gen_simple_segmentation
from acia.viz import render_segmentation
import gzip
import numpy as np

# store segmentation
red_overlay = Overlay([c for c in result if (c.id in df.index and df.loc[c.id]["label_name"] == "e2_crimson")])
blue_overlay = Overlay([c for c in result if (c.id in df.index and df.loc[c.id]["label_name"] == "mvenus")])

# Make a video with
video_file = str(output_path_rel / "filter_segmented.mp4")

# render fluorescent strains in different colors
rendered_seq = render_segmentation(source.to_channel(phase_contrast_channel).to_rgb(), red_overlay, cell_color=(255, 0, 0))
rendered_seq = render_segmentation(rendered_seq, blue_overlay, cell_color=(0, 0, 255))

# after that render the timestamps and scalebar such that they are ontop
rendered_seq = render_time(rendered_seq, **time_config)
rendered_seq = render_scalebar(rendered_seq, **scalebar_config)

render_video(rendered_seq, filename=video_file, **video_config, framerate=framerate)

# Display the rendered segmentation
display(Markdown("# Your segmentation"))

myvideo =  VideoFileClip(video_file)
myvideo.ipython_display(maxduration=400)

# 4. Generating insights into the temporal Co-Culture growth dynamics

We start with the count of cells per frame

In [ ]:
import numpy as np
from sklearn.metrics import r2_score
from scipy.stats import pearsonr


def linear_regression(x,y):
    # linear regression to the log of data
    params, _, _, _, _ = np.polyfit(x, np.log(y), 1, full=True)

    # following exponential model: N(t) = N_0 * np.exp(mu * t)
    N_0 = np.exp(params[1])
    mu = params[0]

    y_pred = N_0 * np.exp(x * mu)

    R2 = r2_score(y, y_pred)

    res = pearsonr(x, np.log(y))

    return N_0, mu, R2, res.statistic

In [ ]:
from scipy import stats

fig, axes = plt.subplots(1, 1, figsize=(7, 7), sharex=True, sharey=True)

colors = ["blue", "red"]
labels = [mvenus_label, e2_crimson_label]
naming = ["mvenus", "e2_crimson"]

for color, label, label_name in zip(colors, labels, naming):

    # sum the area of all individual cells at a certain frame
    df_areas = df[df.label_name==label_name].groupby("time")["area"].agg("sum").reset_index(name="area")

    #print(df_areas)

    t = df_areas["time"]
    y = np.array(df_areas["area"])

    y_diffs = y[1:] - y[0:-1]
    y_log_diffs = np.log(y[1:]) - np.log(y[0:-1])
    print(y_log_diffs)
    #print(len(t), len(y_diffs), len(y))
    res = pearsonr(t[:-1], y_log_diffs)
    print(res)
    r=res[0]
    n = len(y_log_diffs)
    tt = r * np.sqrt((n- 2) / (1- r**2))
    print(tt)
    pval = stats.t.sf(np.abs(tt), n-1)*2
    print(pval)
    print(pearsonr(t[:-1], y_diffs))
    print(pearsonr(t, y))

    A_0, mu, R2, r_xy = linear_regression(t, y)

    # compute the exponential model predictions
    y_pred = A_0 * np.exp(t * mu)

    axes.scatter(t, y, label=f"measured [{label_name}]", color=color, marker="+")
    axes.plot(t, y_pred, label=f"fitted [{label_name}] ($\mu={mu:.2f}$ | $R^2={R2:.4f}$ | $r_{{xy}}={r_xy:.2f}$)", color=color, ls="--")

plt.xlabel("Time [minute]")
plt.ylabel("$\sum$ Total Single-Cell Area [$\mu m^2$]")

plt.legend()
plt.yscale("log")
plt.grid(True)

plt.xlim((0, None))



In [ ]:
import matplotlib.gridspec as gridspec
import seaborn as sns
import pandas as pd

gs = gridspec.GridSpec(4,1, hspace=0)
fig = plt.figure(figsize=(9, 10))

#first plot
ax = fig.add_subplot(gs[0])

growth_estimates = []

for color, label, label_name in zip(colors, labels, naming):

    # sum the area of all individual cells at a certain frame
    df_areas = df[df.label_name==label_name].groupby("time")["area"].agg("sum").reset_index(name="area")

    #print(df_areas)

    t = df_areas["time"]
    y = df_areas["area"]

    A_0, mu, R2, r_xy = linear_regression(t, y)

    # compute the exponential model predictions
    y_pred = A_0 * np.exp(t * mu)

    ax.scatter(t, y, label=f"measured [{label_name}]", color=color, marker="+")
    ax.plot(t, y_pred, label=f"fitted [{label_name}] ($\mu={mu:.2f}$ | $R^2={R2:.4f}$ | $r_{{xy}}={r_xy:.2f}$)", color=color, ls="--")

    growth_estimates.append(dict(
        image_id = image_id.stem,
        label_name = label_name,
        A_0 = A_0,
        mu = mu,
        R2 = R2
    ))

pd.DataFrame(growth_estimates).to_csv(output_path / "growth_estimates.csv")


ax.set_ylabel("$\sum$ Total Single-Cell Area [$\mu m^2$]")

ax.legend()
ax.set_yscale("log")
ax.grid(True)

#ax.set_xlim((0, None))
#ax.tick_params(
#    axis='x',          # changes apply to the x-axis
#    labelbottom='off') # labels along the bottom edge are off
#ax.axes.xaxis.set_ticklabels([])

#second plot
ax2 = fig.add_subplot(gs[1], sharex=ax)
ax2.set_ylabel("Average cell size\n[$\mu m^2$]", size=12)
sns.lineplot(data=df[df.label==mvenus_label], x="time", y="area", color=colors[0])
ax2.grid(True)

ax3 = fig.add_subplot(gs[2], sharey=ax2, sharex=ax)
ax3.set_ylabel("Average cell size\n[$\mu m^2$]", size=12)
sns.lineplot(data=df[df.label==e2_crimson_label], x="time", y="area", color=colors[1])
ax3.grid(True)
ax3.set_xlabel("Time [hour]")

ax4 = fig.add_subplot(gs[3], sharex=ax)
ax4.set_ylabel("Average fluorescence\n[a.u.]", size=12)
sns.lineplot(data=df[df.label==mvenus_label], x="time", y="mvenus", color=colors[0])
sns.lineplot(data=df[df.label==e2_crimson_label], x="time", y="e2-crimson", color=colors[1])
ax4.grid(True)
ax4.set_xlabel("Time [hour]")

fig.suptitle(f"Co-culture Fluorescence Analysis Summary\n[image_id={image_id.stem}]", fontsize=16)

plt.savefig(output_path / "summary_vertical.svg")
plt.savefig(output_path / "summary_vertical.png", dpi=300)


In [ ]:
df[df.label==mvenus_label]

In [ ]:
df.to_csv(output_path / "allcells.csv")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(9, 3))

axes[0].imshow(rendered_seq.get_frame(8).raw)
axes[1].imshow(rendered_seq.get_frame(20).raw)
axes[2].imshow(rendered_seq.get_frame(32).raw)

axes[0].axis("off")
axes[1].axis("off")
axes[2].axis("off")

plt.tight_layout()

plt.savefig(output_path / "coculture_images.png", dpi=300)

In [ ]:
import cv2
im1 = cv2.imread(str(output_path / "coculture_images.png"))
im2 = cv2.imread(str(output_path / "summary_vertical.png"))

fig, axes = plt.subplots(2, 1, gridspec_kw={'height_ratios': [1, 3.9]}, figsize=(10, 15))

axes[0].imshow(cv2.cvtColor(im1, cv2.COLOR_RGB2BGR))
axes[1].imshow(cv2.cvtColor(im2, cv2.COLOR_RGB2BGR))
axes[0].axis("off")
axes[1].axis("off")

plt.tight_layout()

plt.savefig(output_path / "complete_summary.png", dpi=300)

## 🔁 Reproducibility Information

pip and conda environment details

In [ ]:
%pip freeze

In [ ]:
%mamba env export